In [16]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster

config_dict = {
    "Building_Id": "57f7cbb0-5996-4aff-a20e-18c2bf51f2aa", #TODO make bldg name
    "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
    "retrofit_scenario": "baseline",
    "schedules_scenaro": "baseline",
    "lab_scenario": "baseline",
    # "retrofit_scenario":  "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\240404_test_templates.csv",
    "TYPOLOGY": "Residential",
    "AGE_ERA": 1980,
    "lab_weight": 0.8
}

def run_single_building(config_dict, output_dir, resource_dir, eplus_loc): #TODO
    weights_df = pd.read_json("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\umi\\shoebox-weights-newpaths.json")
    
    weights_df = weights_df[weights_df.ParentBuildingId == config_dict["Building_Id"]]

    idfs = set(weights_df.ShoeboxPath.to_list())
    # Mutations
    features_df = pd.read_csv(config_dict["retrofit_scenario"])

    with open("tests/template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open("tests/schedule_bunches.json", "r") as f:
        schedules = json.load(f)

    new_idfs = []

    for idf in idfs:
        try:
            epjson = EpJsonIDF.from_epjson(Path(idf).with_suffix(".epJSON"), eplus_loc=eplus_loc)
            features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
            features_series = features_series.iloc[0]

            cols = set(features_series.index) & set(features_map.keys())

            epjson.replace_features(features_series[cols], features_map)
            # epjson.remove("ZoneVentilation:DesignFlowRate")
            epjson.update_schedules(schedules)
            epjson.add([(
                "Output:SQLite",
                {"Output:SQLite 1": {"option_type": "Simple"}}
                )])
            epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                {"air_changes_per_hour": 0}
            )
            # epjson.save()
            epjson.save_idf(output_path=output_dir) #suffix="_test",
            new_idfs.append(output_dir / Path(idf))
            print("Completed", Path(idf))

        except Exception as e:
            print(f"FAILED ON {idf}")
            raise e
    
    idf_cluster = SimulateCluster(
        idf_list = new_idfs,
        epw = config_dict["epw"],
        weights_df = weights_df,
        eplus_location=eplus_loc
    )

    errors = idf_cluster.parallel_simulate()
    r = idf_cluster.fetch_building_results_parallel([config_dict["Building_Id"]])

    return r, errors

In [36]:
res, err = run_single_building(config_dict, output_dir=os.getcwd())

C:\Users\zoelh\AppData\Local\Temp\ipykernel_488684\1151875029.py:25: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  epjson.replace_features(features_series[cols], features_map)
INFO:EPJSON:Setting VentilationMode to OccupancySchedule
INFO:EPJSON:Setting OASchedule to 30HrWk
INFO:EPJSON:Setting CoolingAvailability to AllOn
INFO:EPJSON:Setting HeatingAvailability to AllOn
INFO:EPJSON:Setting RoofRValue to 2.591349481
INFO:EPJSON:Setting LightsSchedule to 30HrWk
INFO:EPJSON:Setting SlabRValue to 2.591349481
INFO:EPJSON:Setting WindowUValue to 1.5
INFO:EPJSON:Setting RecoverySettings to Enthalpy
INFO:EPJSON:Setting EquipmentSchedule to 60HrWk
INFO:EPJSON:Setting VentilationPerPerson to 0.000794118
INFO:EPJSON:Setting FacadeRValue to 1.17266436
INFO:EPJSON:Setting OccupancySchedule to 60HrWk
INFO:EPJSON:Setting InfiltrationType to AirChanges/Hour
INFO:EPJSON:Setting Infiltration to 1.823529412
INFO:EPJSON:Setting Ventilatio

Completed D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE1\15\SAMPLE1.idf


INFO:EPJSON:Setting VentilationPerArea to 0.000794118
INFO:EPJSON:Setting DHWSupplyTemp to 65.0
INFO:EPJSON:Setting EquipmentPowerDensity to 10.0
INFO:EPJSON:Setting PeopleDensity to 0.15
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:Setting LightingPowerDensity to 10.0
INFO:EPJSON:Setting CoolingSetpoint to 23.64705882
INFO:EPJSON:Setting LatentRecoveryEffectiveness to 0.65
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting HeatingSetpoint to 23.64705882
INFO:EPJSON:Setting DHWFlowRate to 3.77e-05
INFO:EPJSON:Setting WindowSHGC to 0.82
INFO:EPJSON:Setting EconomizerSettings to DifferentialEnthalpy
INFO:EPJSON:Updating definition of Output:SQLite
INFO:EPJSON:Building idf for D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE0\15\SAMPLE0.epJSON
INFO:EPJSON:Converting D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE0\15\SAMPLE0.epJSON to idf
INFO:EPJSON:Ene

Completed D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE0\15\SAMPLE0.idf


INFO:EPJSON:Setting VentilationPerArea to 0.000794118
INFO:EPJSON:Setting DHWSupplyTemp to 65.0
INFO:EPJSON:Setting EquipmentPowerDensity to 10.0
INFO:EPJSON:Setting PeopleDensity to 0.15
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:Setting LightingPowerDensity to 10.0
INFO:EPJSON:Setting CoolingSetpoint to 23.64705882
INFO:EPJSON:Setting LatentRecoveryEffectiveness to 0.65
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting HeatingSetpoint to 23.64705882
INFO:EPJSON:Setting DHWFlowRate to 3.77e-05
INFO:EPJSON:Setting WindowSHGC to 0.82
INFO:EPJSON:Setting EconomizerSettings to DifferentialEnthalpy
INFO:EPJSON:Updating definition of Output:SQLite
INFO:EPJSON:Building idf for D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE3\15\SAMPLE3.epJSON
INFO:EPJSON:Converting D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE3\15\SAMPLE3.epJSON to idf
INFO:EPJSON:Ene

Completed D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE3\15\SAMPLE3.idf


INFO:EPJSON:Setting DHWSupplyTemp to 65.0
INFO:EPJSON:Setting EquipmentPowerDensity to 10.0
INFO:EPJSON:Setting PeopleDensity to 0.15
INFO:EPJSON:Setting SensibleRecoveryEffectiveness to 0.7
INFO:EPJSON:Setting LightingPowerDensity to 10.0
INFO:EPJSON:Setting CoolingSetpoint to 23.64705882
INFO:EPJSON:Setting LatentRecoveryEffectiveness to 0.65
INFO:EPJSON:Setting PartitionRValue to 0.227857143
INFO:EPJSON:Setting HeatingSetpoint to 23.64705882
INFO:EPJSON:Setting DHWFlowRate to 3.77e-05
INFO:EPJSON:Setting WindowSHGC to 0.82
INFO:EPJSON:Setting EconomizerSettings to DifferentialEnthalpy
INFO:EPJSON:Updating definition of Output:SQLite
INFO:EPJSON:Building idf for D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE2\15\SAMPLE2.epJSON
INFO:EPJSON:Converting D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE2\15\SAMPLE2.epJSON to idf
INFO:EPJSON:EnergyPlus Starting
INFO:EPJSON:EnergyPlus, Version 22.2.

Completed D:\Users\zoelh\Dropbox (MIT)\4.S42 Campus Decarb\Energy Modeling\idfs\umi\eplus\Group16\SAMPLE2\15\SAMPLE2.idf


100%|██████████| 4/4 [00:00<?, ?it/s]
INFO:BATCH:Simulated all idfs.
INFO:BATCH:Fetching results for 57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
INFO:BATCH:There are 4 shoeboxes for 57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
Fetching building results: 100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


In [37]:
res['57f7cbb0-5996-4aff-a20e-18c2bf51f2aa']

,Zone Lights Electricity Energy,Zone Electric Equipment Electricity Energy,Zone Ideal Loads Supply Air Total Heating Energy,Zone Ideal Loads Supply Air Total Cooling Energy,Water Use Equipment Heating Energy,Zone Lights Electricity Energy_norm,Zone Electric Equipment Electricity Energy_norm,Zone Ideal Loads Supply Air Total Heating Energy_norm,Zone Ideal Loads Supply Air Total Cooling Energy_norm,Water Use Equipment Heating Energy_norm,ModelArea,DateTime,BuildingId
0,224.856325,1030.591488,396.795824,34.021444,0.00000,0.011966,0.054846,0.021117,0.001811,0.000000,18790.563417,2018-01-01 00:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
1,224.856325,1030.591488,281.060182,106.255748,0.00000,0.011966,0.054846,0.014958,0.005655,0.000000,18790.563417,2018-01-01 01:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
2,224.856325,1030.591488,287.393147,93.234659,0.00000,0.011966,0.054846,0.015295,0.004962,0.000000,18790.563417,2018-01-01 02:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
3,224.856325,1030.591488,289.711600,85.807250,0.00000,0.011966,0.054846,0.015418,0.004567,0.000000,18790.563417,2018-01-01 03:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
4,224.856325,1030.591488,287.168566,86.712875,0.00000,0.011966,0.054846,0.015283,0.004615,0.000000,18790.563417,2018-01-01 04:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,224.856325,1030.591488,172.455112,245.149122,77.81711,0.011966,0.054846,0.009178,0.013046,0.004141,18790.563417,2018-12-31 19:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
8756,224.856325,1030.591488,169.658463,254.570365,0.00000,0.011966,0.054846,0.009029,0.013548,0.000000,18790.563417,2018-12-31 20:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
8757,224.856325,1030.591488,176.254930,246.236600,0.00000,0.011966,0.054846,0.009380,0.013104,0.000000,18790.563417,2018-12-31 21:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
8758,224.856325,1030.591488,168.524914,259.683327,0.00000,0.011966,0.054846,0.008969,0.013820,0.000000,18790.563417,2018-12-31 22:00:00,57f7cbb0-5996-4aff-a20e-18c2bf51f2aa
